In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%pip install datasets vllm

In [3]:
import os
import random
from datasets import load_dataset
from transformers import AutoTokenizer
from vllm import LLM, SamplingParams

## Load model and tokenizer

In [4]:
# "solidrust/Phi-3-mini-128k-instruct-AWQ"
model_card = "CohereForAI/aya-23-8B"
tokenizer = AutoTokenizer.from_pretrained(model_card)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
# Define model
model = LLM(
    model=model_card,
    # quantization='awq', # It is not necessary if we use the AYA model
    enable_prefix_caching=False,
    tensor_parallel_size=1,
    gpu_memory_utilization=0.8,
)


# Prompt and params
sampling_params = SamplingParams(
    temperature=0.5,
    top_p=0.95,
    max_tokens=5000,
)

INFO 07-20 14:28:21 llm_engine.py:174] Initializing an LLM engine (v0.5.2) with config: model='CohereForAI/aya-23-8B', speculative_config=None, tokenizer='CohereForAI/aya-23-8B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None), seed=0, served_model_name=CohereForAI/aya-23-8B, use_v2_block_manager=False, enable_prefix_caching=False)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


WARNING 07-20 14:28:24 interfaces.py:131] The model (<class 'vllm.model_executor.models.commandr.CohereForCausalLM'>) contains all LoRA-specific attributes, but does not set `supports_lora=True`.
INFO 07-20 14:28:24 weight_utils.py:218] Using model weights format ['*.safetensors']
INFO 07-20 14:28:29 model_runner.py:266] Loading model weights took 14.9553 GB
INFO 07-20 14:28:34 gpu_executor.py:86] # GPU blocks: 7054, # CPU blocks: 2048
INFO 07-20 14:28:36 model_runner.py:1007] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 07-20 14:28:36 model_runner.py:1011] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 07-20 14:28:53 model_runner.py:1208]

## Load the dataset

In [6]:
dataset_card =  "wikimedia/wikipedia"
dataset = load_dataset(dataset_card, "20231101.es")

# Take Spanish train
spanish_dataset = dataset['train']
spanish_dataset

Dataset({
    features: ['id', 'url', 'title', 'text'],
    num_rows: 1841155
})

## Shuffle and subsample the dataset to get 100,000 entries

In [7]:
random.seed(42)
indices = list(range(len(spanish_dataset)))
random.shuffle(indices)
subsample_indices = indices[:100000]
subsample_spanish_dataset = spanish_dataset.select(subsample_indices)
subsample_spanish_dataset

Dataset({
    features: ['id', 'url', 'title', 'text'],
    num_rows: 100000
})

## Create prompts by entry

In [8]:
def generate_prompt(text):
  system_prompt = 'Eres un asistente de IA experto en realizar resúmenes en español dado un texto.'
  user_prompt = f'''
  Genera un resumen en lenguaje natural del siguiente artículo de Wikipedia en un párrafo de entre 4 y 6 líneas:
  {text}
  IMPORTANTE: Inicia la respuesta con el resumen del texto.
  '''

  chat = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
  ]
  return tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)

## Save the output in a text file

In [9]:
file_path = "/content/drive/MyDrive/Colab/NER/wikipedia_summary.txt"

In [10]:
def save_text(texts):
    with open(file_path, "a", encoding='utf-8') as file:
    for text in texts:
        file.write(text + "\n")
    print(f"- Saved {len(texts)} texts")

## Process end to end

In [11]:
if os.path.exists(file_path):
    os.remove(file_path)
    print("- Archivo eliminado exitosamente.")

chunk_subsample = 10000
chunk_size = 2000

for j in range(0, len(subsample_spanish_dataset), chunk_subsample):
  chunk_dataset = subsample_spanish_dataset.select(range(j, min(j + chunk_subsample, len(subsample_spanish_dataset))))
  print(f"- Processing chunk {j} to {j + chunk_subsample}...")

  for i in range(0, len(chunk_dataset), chunk_size):
    chunk = chunk_dataset.select(range(i, min(i + chunk_size, len(chunk_dataset))))
    print(f"- Processing chunk subset {j}, chunk {i} to {i + chunk_size}...")

    chunk_texts = []
    prompts = []

    # process chunk
    for data in chunk:
      prompts.append(generate_prompt(data['text']))

    # Generate summaries
    print("- Generating summaries...")
    summary_output = model.generate(prompts=prompts, sampling_params=sampling_params)

    # Extract text
    for item in summary_output:
      text = item.outputs[0].text
      chunk_texts.append(text)

    print("- Saving texts...")
    save_text(chunk_texts)

- Processing chunk 0 to 10000...
- Processing chunk subset 0, chunk 0 to 2000...
- Generating summaries...


Processed prompts:   0%|          | 0/2000 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

WARNING 07-20 14:29:36 scheduler.py:1124] Sequence group 144 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_preemption=1


Processed prompts:   3%|▎         | 55/2000 [00:17<03:54,  8.28it/s, est. speed input: 1623.94 toks/s, output: 301.78 toks/s]

WARNING 07-20 14:29:45 scheduler.py:698] Input prompt (12389 tokens) is too long and exceeds limit of 8192


...

WARNING 07-20 18:34:42 scheduler.py:698] Input prompt (15747 tokens) is too long and exceeds limit of 8192


Processed prompts:  86%|████████▌ | 1717/2000 [03:52<00:31,  8.86it/s, est. speed input: 5972.80 toks/s, output: 1042.83 toks/s]

WARNING 07-20 18:35:06 scheduler.py:698] Input prompt (16633 tokens) is too long and exceeds limit of 8192


Processed prompts: 100%|██████████| 2000/2000 [04:28<00:00,  7.44it/s, est. speed input: 6077.18 toks/s, output: 1065.29 toks/s]

- Saving texts...
- Saved 2000 texts
